In [81]:
import tensorflow as tf

In [82]:
def filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    # Compute box scores
    box_scores = box_confidence * box_class_probs
    
    # Find the box_classes thanks to the max box_scores, keep track of the corresponding score
    box_classes = tf.argmax(box_scores, axis=-1)
    box_class_scores = tf.reduce_max(box_scores, axis=-1)
    
    # Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    filtering_mask = (box_class_scores >= threshold)
    
    # Apply the mask to scores, boxes and classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [83]:
def box_area(box):
    y1 = box[0]; x1 = box[1]; y2 = box[2]; x2 = box[3]    
    return tf.maximum(x2 - x1, 0) * tf.maximum(y2 - y1, 0)

def iou(box1, box2):
    # Calculate the (y1, x1, y2, x2) coordinates of the intersection of box1 and box2.
    y1 = tf.maximum(box1[0], box2[0])
    x1 = tf.maximum(box1[1], box2[1])
    y2 = tf.minimum(box1[2], box2[2])
    x2 = tf.minimum(box1[3], box2[3])
    intersection_box = (y1, x1, y2, x2)
    inter_area = box_area(intersection_box)

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    union_area = box_area(box1) + box_area(box2) - inter_area
    
    # Compute the IoU
    iou = inter_area / union_area
    
    return iou

In [84]:
def non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')
    tf.variables_initializer([max_boxes_tensor])
    
    selected_indices = tf.image.non_max_suppression(
        boxes,
        scores,
        max_boxes,
        iou_threshold
    )
        
    scores = tf.gather(scores, selected_indices)
    boxes = tf.gather(boxes, selected_indices)
    classes = tf.gather(classes, selected_indices)
    
    return scores, boxes, classes

In [85]:
def yolo_boxes_to_corners(box_xy, box_wh):
    box_xy2 = box_xy + box_wh / 2.
    box_xy1 = box_xy - box_wh / 2.
    box_xy_wh = tf.concat([
        box_xy1[..., 1:2],
        box_xy1[..., 0:1], 
        box_xy2[..., 1:2],
        box_xy2[..., 0:1]
    ], -1)
    return box_xy_wh


def scale_boxes(boxes, image_shape):
    image_shape_4d = [image_shape[0], image_shape[1], image_shape[0], image_shape[1]]
    image_shape_tensor = tf.Variable(image_shape_4d, dtype='float')
    tf.variables_initializer([image_shape_tensor])
    return tf.multiply(boxes,image_shape_4d)

    
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    # Retrieve outputs of the YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    
    # Convert boxes to be ready for filtering functions 
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    # Use one of the functions you've implemented to perform Score-filtering with a threshold of score_threshold
    scores, boxes, classes = filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)
        
    # Use one of the functions you've implemented to perform Non-max suppression with a threshold of iou_threshold
    scores, boxes, classes = non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
    
    return scores, boxes, classes
    